<a href="https://colab.research.google.com/github/Kirilenkov/docx_semantic_similarity/blob/main/Cozyduke.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Сервис автоматического анализа текста двух документов и присвоения статуса соответсвия
<hr>


Над проектом работает:
Команда **'COZYDUKE'.**

0. Кирсанов Вадим Вадимович
1. Казаченко Екатерина Александровна
2. Кириленков Кирилл Владимирович
3. Гайсина Лиана Ильдаровна
4. Федеряев Клим Александрович
5. Бадретдинова Рушания Ринатовна



In [46]:
# @title Дополнительные модули
# Установка дополнительных модулей
%pip install python-docx
%pip install POT
%pip install torch

# Импорт библиотек и их модулей
import re
from io import BytesIO

import docx
import gensim.downloader as api
import numpy as np
import pandas as pd
import requests
import torch
from gensim.models import KeyedVectors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity



# 1. Подготовка данных
* **Извлечение данных:** Данные были извлечены из файлов в формате Excel (для анализа требований) и DOCX (для анализа технических характеристик). Содержимое DOCX файла, включающее тексты параграфов и заголовки, было загружено с использованием библиотеки python-docx.

* **Предобработка текста**: Произведена предобработка текстовых данных, включая удаление знаков пунктуации, приведение всех букв к нижнему регистру и удаление лишних пробелов. Эти шаги были выполнены с помощью регулярных выражений и методов обработки строк в Python.
* **Создание словарей и датафреймов:** На основе данных из Excel файла был создан словарь требований. Также был составлен датафрейм для хранения информации о параграфах из DOCX файла, что обеспечило удобную структуру для последующего анализа.

**Примечание**: В дальнейшем планируется улучшить предобработку текста. А также доработать код для извлечения информации из каждого пункта документа.

In [47]:
# @title Создаем словарь с требованиями из файла exel.
# @title Для корректного выполнения кода вы можете заменить путь к файлу на свой в переменной file_path

# Загрузка файла Excel
file_url = 'https://docs.google.com/spreadsheets/d/18wM88vqURx4rKCaeQuf5BTbAF9Uvr2T0/export?format=xlsx'
response = requests.get(file_url)
data = pd.read_excel(BytesIO(response.content))

# Удаление переноса строк, лишних пробелов, приведение к нижнему регистру и удаление знаков пунктуации
data['Формулировка в СТО ИНТИ'] = data['Формулировка в СТО ИНТИ'].apply(
    lambda x: re.sub(r'\s+', ' ', re.sub(r'[^\w\s]', '', str(x).lower()).replace('\n', ' ').strip())
)

# Выделение первых двух колонок и их предобработка
data_clean = data[['Пункт СТО ИНТИ', 'Формулировка в СТО ИНТИ']]

# Создание словаря
items_dict = data_clean.set_index('Пункт СТО ИНТИ')['Формулировка в СТО ИНТИ'].to_dict()

# Вывод первых пяти элементов словаря для проверки
list(items_dict.items())[:5]


[('6.1.1',
  'насосное оборудование включая вспомогательные системы на которое распространяется настоящий стандарт должно конструироваться и изготавливаться в расчете на срок службы не менее 20 лет за исключением естественно изнашиваемых деталей согласно таблице 14 и не менее 3 лет непрерывной эксплуатации остановка оборудования для выполнения техобслуживания или проверки не является нарушением этого требования'),
 ('6.1.3.7',
  'поскольку в герметичных насосах для охлаждения и смазки подшипников используется перекачиваемая среда или другая жидкость она должна оставаться стабильной при прохождении через подшипники поставщик должен обеспечить работоспособность с учетом изменения температуры и давления жидкости циркулирующей в насосе и полости ротора работоспособность должна быть обеспечена при максимальной заданной рабочей температуре для минимального стабильного расхода нормальных рабочих условий и максимального номинального расхода поставщик должен также предоставить npshr любых вспом

In [48]:
# @title Создаем датафрейм с техническими характеристиками по абзацам

# Загрузка документа Word
docx_url = 'https://docs.google.com/document/d/1iJPrBdchU1ZeXv-oW9cokJP1GN72WDGE/export?format=docx'

response = requests.get(docx_url)
doc = docx.Document(BytesIO(response.content))

# Чтение всех параграфов в документе, добавление их в список с идентификацией номера пункта или заголовка
all_paragraphs = []
current_header = "Без заголовка"  # Начальный заголовок для текста без явного разделения

for para in doc.paragraphs:
    if para.text.strip():  # Проверка на непустой текст
        # Ищем пункты или заголовки, определяем по форматированию или содержимому
        if para.style.name.startswith('Heading') or (para.text.split()[0].replace('.', '').isdigit() and len(para.text.split()) > 1):
            current_header = para.text.split(maxsplit=1)[0] if para.text.split()[0].replace('.', '').isdigit() else para.text
        all_paragraphs.append((current_header, para.text))

# Создание DataFrame со всем текстом и заголовками
df_all_text = pd.DataFrame(all_paragraphs, columns=['Заголовок', 'Текст параграфа'])

# Предобработка текста: приведение к нижнему регистру, удаление знаков пунктуации и лишних пробелов
df_all_text['Текст параграфа'] = df_all_text['Текст параграфа'].apply(
    lambda x: re.sub(r'[^\w\s]', '', x.lower()).replace('\n', ' ').strip())
df_all_text['Текст параграфа'] = df_all_text['Текст параграфа'].apply(
    lambda x: re.sub(r'\s+', ' ', x))

# Вывод первых пяти строк обработанного DataFrame для проверки
df_all_text.head(40)

,Заголовок,Текст параграфа
0,Без заголовка,содержание
1,Без заголовка,настоящие технические условия распространяются...
2,Без заголовка,агрегаты не предназначены для перекачивания кр...
3,Без заголовка,допускается перекачивание жидкостей с температ...
4,Без заголовка,агрегаты изготавливаются в климатическом испол...
5,Без заголовка,в условиях умеренного климата у2 45 40 с
6,Без заголовка,в условиях умеренного и холодного климата ухл1...
7,Без заголовка,в условиях холодного климата хл1хл2 6040 с
8,Без заголовка,в условиях умереннохолодного и тропического мо...
9,Без заголовка,допустимый диапазон температуры окружающей сре...


In [49]:
# Разделение строки "Заголовок" на номер пункта и чистый заголовок, если номер пункта присутствует

# Добавление нового столбца для номера пункта
df_all_text['Номер пункта'] = None

# Разбиение текущего заголовка на возможный номер пункта и остальную часть
for index, row in df_all_text.iterrows():
    header_text = row['Заголовок']
    # Проверяем, начинается ли заголовок с числовой последовательности, которая может быть номером пункта
    if header_text.split()[0].replace('.', '').isdigit():
        # Извлекаем номер пункта и обновляем заголовок
        split_header = header_text.split(maxsplit=1)
        df_all_text.at[index, 'Номер пункта'] = split_header[0]
        df_all_text.at[index, 'Заголовок'] = split_header[1] if len(split_header) > 1 else ""  # Если есть что-то после номера пункта

# Вывод обновленного DataFrame с отдельным столбцом для номера пункта
df_all_text.head()

,Заголовок,Текст параграфа,Номер пункта
0,Без заголовка,содержание,None
1,Без заголовка,настоящие технические условия распространяются...,None
2,Без заголовка,агрегаты не предназначены для перекачивания кр...,None
3,Без заголовка,допускается перекачивание жидкостей с температ...,None
4,Без заголовка,агрегаты изготавливаются в климатическом испол...,None


In [50]:
# Фильтрация DataFrame для поиска строк с определенными номерами пунктов
df_with_numbers = df_all_text[df_all_text['Номер пункта'].notna()]

# Вывод строк с номерами пунктов
df_with_numbers.head()


,Заголовок,Текст параграфа,Номер пункта
177,,357 испытания головного образца агрегата элект...,3.5.7
178,,испытания головных образцов агрегатов электрон...,3.5.7
179,,проверки прочности ротора насоса при критическ...,3.5.7
180,,гидравлических испытаний на прочность корпусов...,3.5.7
181,,проверки работоспособности агрегатов при возде...,3.5.7


# 2. Векторизация текста и расчёт сходства
* **Векторизация текста**: Используем TF-IDF векторизацию для преобразования текстов требований и параграфов в числовые векторы и подготовки данных к анализу сходства.

* **Расчет косинусного сходства**: Вычисляем косинусное сходство между векторами параграфов и требований для определения степени соответствия между ними.

In [51]:
# Подготовка текста: соберем все тексты параграфов в список
texts = df_all_text['Текст параграфа'].tolist()

# Подготовка списка текстов требований из предыдущей предобработки (словаря требований)
requirement_texts = list(items_dict.values())

# Объединение текстов требований и текстов параграфов для обучения одного TF-IDF векторизатора
combined_texts = texts + requirement_texts

# Создание и обучение TF-IDF векторизатора на объединенном корпусе
combined_tfidf_vectorizer = TfidfVectorizer()
combined_tfidf_matrix = combined_tfidf_vectorizer.fit_transform(combined_texts)

# Разделение матрицы TF-IDF на часть параграфов и часть требований
tfidf_paragraphs = combined_tfidf_matrix[:len(texts), :]
tfidf_requirements = combined_tfidf_matrix[len(texts):, :]

# Расчет косинусного сходства между параграфами и требованиями
cosine_similarities = cosine_similarity(tfidf_paragraphs, tfidf_requirements)

# Вывод формы матрицы сходств для проверки
cosine_similarities.shape


(268, 16)

#3. Определение наилучших совпадений
* **Анализ сходства:** Для каждого параграфа определяем требование с максимальным уровнем сходства и выявляем наиболее релевантные соответствия.
* **Создание итогового отчета:** Результаты анализа собраем в датафрейме, который включает информацию о параграфах, соответствующих требованиях и степени их сходства.

In [52]:
# Нахождение индекса требования с максимальным сходством для каждого параграфа
max_similarity_indices = np.argmax(cosine_similarities, axis=1)

# Создание списка для отображения наивысшего соответствия для каждого параграфа
best_match_info = []
for i, index in enumerate(max_similarity_indices):
    paragraph_text = texts[i]
    best_requirement_text = requirement_texts[index]
    similarity_score = cosine_similarities[i, index]
    best_match_info.append((paragraph_text[:50] + '...', best_requirement_text[:50] + '...', similarity_score))

# Пример вывода для первых 5 параграфов
best_match_info[:5]

[('содержание...',
  'насосное оборудование включая вспомогательные сист...',
  0.0),
 ('настоящие технические условия распространяются на ...',
  'подшипники и корпуса подшипников с технологическим...',
  0.046189428551539076),
 ('агрегаты не предназначены для перекачивания криста...',
  'насосы должны быть предназначены для работы с легк...',
  0.19023786044935814),
 ('допускается перекачивание жидкостей с температурой...',
  'насосы работающие совместно с регулируемым приводо...',
  0.08448349175237255),
 ('агрегаты изготавливаются в климатическом исполнени...',
  'насос и его привод должны удовлетворять критериям ...',
  0.049962474002380027)]

In [53]:
# Создание DataFrame с наилучшими совпадениями для каждого требования
best_match_df = pd.DataFrame(columns=['Номер требования', 'Текст требования', 'Заголовок параграфа', 'Пункт параграфа', 'Текст параграфа', 'Сходство'])

# Перебор каждого требования и нахождение параграфа с максимальным сходством
for req_index, req_text in enumerate(requirement_texts):
    # Индекс параграфа с максимальным сходством для текущего требования
    paragraph_index = np.argmax(cosine_similarities[:, req_index])
    # Соответствующий максимальный сходственный параграф
    max_similarity = cosine_similarities[paragraph_index, req_index]
    # Данные параграфа
    paragraph_text = df_all_text.loc[paragraph_index, 'Текст параграфа']
    paragraph_header = df_all_text.loc[paragraph_index, 'Заголовок']
    paragraph_number = df_all_text.loc[paragraph_index, 'Номер пункта']

# Инициализация пустого DataFrame
best_match_df = pd.DataFrame(columns=['Пункт требования', 'Текст требования', 'Заголовок параграфа', 'Пункт параграфа', 'Текст параграфа', 'Сходство'])

# Сбор всех строк для добавления
rows_to_add = []
for req_key, req_text in items_dict.items():
    paragraph_index = np.argmax(cosine_similarities[:, list(items_dict.keys()).index(req_key)])
    max_similarity = cosine_similarities[paragraph_index, list(items_dict.keys()).index(req_key)]
    paragraph_text = df_all_text.loc[paragraph_index, 'Текст параграфа']
    paragraph_header = df_all_text.loc[paragraph_index, 'Заголовок']
    paragraph_number = df_all_text.loc[paragraph_index, 'Номер пункта']

    # Создание временного DataFrame для текущей строки
    temp_df = pd.DataFrame({
        'Пункт требования': [req_key],
        'Текст требования': [req_text],
        'Заголовок параграфа': [paragraph_header],
        'Пункт параграфа': [paragraph_number],
        'Текст параграфа': [paragraph_text],
        'Сходство': [max_similarity]
    })
    rows_to_add.append(temp_df)

# Добавление всех собранных строк в основной DataFrame
best_match_df = pd.concat(rows_to_add, ignore_index=True)

# Вывод первых нескольких строк для проверки
best_match_df.head(16)



,Пункт требования,Текст требования,Заголовок параграфа,Пункт параграфа,Текст параграфа,Сходство
0,6.1.1,насосное оборудование включая вспомогательные ...,Требования надежности,None,установленный срок службы 30 лет срок службы а...,0.225084
1,6.1.3.7,поскольку в герметичных насосах для охлаждения...,Основные параметры и характеристики,None,максимальное допускаемое давление на входе в н...,0.127161
2,6.1.5,насосы должны быть предназначены для работы с ...,Без заголовка,None,агрегаты не предназначены для перекачивания кр...,0.190238
3,6.1.6,конструкция насоса должна предусматривать возм...,Конструктивные особенности,None,с несколькими секциями с целью повышения напор...,0.171901
4,6.1.7,насосы должны сохранять свою работоспособность...,Основные параметры и характеристики,None,б не менее 105 от номинальной частоты вращения...,0.830750
5,6.1.8,насосы работающие совместно с регулируемым при...,Основные параметры и характеристики,None,насосы работающие совместно с регулируемым при...,0.981380
6,6.1.25.2,если место установки электроопасное электродви...,,7.15,715 электрооборудование должно соответствовать...,0.147772
7,6.1.27,насос и его привод должны удовлетворять критер...,Правила приемки,None,параметры и характеристики контролируемые при ...,0.180913
8,6.1.31.1,резьбовые детали должны соответствовать гост 8...,,7.16,716 требования безопасности агрегатов должны с...,0.174960
9,6.1.31.3,при применении резьб по гост 8724 исо 261 и 9 ...,"Требования к материалам, покупным изделиям и и...",None,концы болтов и шпилек должны выступать из гаек...,0.311947


#4. Изучение альтернативных способов определения сходства

* **Использование модели BERT**: Для начала, был загружен предобученный токенизатор и модель BERT (bert-base-uncased) из библиотеки Hugging Face. Эта модель является одной из базовых версий BERT, предназначенной для работы с текстами.

* **Работа с текстами**
Списки текстов из параграфов и требований были подготовлены для обработки. Тексты были извлечены из датафрейма best_match_df, который содержал информацию о наилучшем соответствии между параграфами и требованиями.Для каждого списка текстов были получены соответствующие эмбеддинги. С использованием функции косинусного сходства было рассчитано соответствие между параграфами и требованиями.

* Однако полученные результаты оказались чересчур оптимистичными и  малоинформативными.

In [ ]:

# Загрузка предобученного токенизатора и модели BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)

def get_bert_embeddings(texts):
    """Функция для получения BERT эмбеддингов."""
    encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=512)
    with torch.no_grad():
        outputs = model(**encoded_input)
    # Используем [CLS] токен для представления всего текста
    embeddings = outputs.last_hidden_state[:, 0, :].numpy()
    return embeddings

# Подготовка текстов
texts = best_match_df['Текст параграфа'].tolist()
requirements = best_match_df['Текст требования'].tolist()

# Получение эмбеддингов
paragraph_embeddings = get_bert_embeddings(texts)
requirement_embeddings = get_bert_embeddings(requirements)

# Расчет косинусного сходства
cosine_sim = cosine_similarity(paragraph_embeddings, requirement_embeddings)

# Сохранение сходства в DataFrame
for idx in range(len(best_match_df)):
    best_match_df.at[idx, 'Сходство BERT'] = cosine_sim[idx, idx]  # диагональные элементы, если порядок соответствует

# Вывод первых строк DataFrame для проверки
best_match_df.head(20)


* **Загрузка модели Word2Vec:** Использовалась предобученная модель Word2Vec ('word2vec-google-news-300') из коллекции Google News, загруженная через API gensim.

* **Подготовка модели:** После загрузки убедились, что модель доступна как объект KeyedVectors для использования функции wmdistance, которая позволяет вычислять расстояние по методу Word Mover's Distance (WMD) между двумя текстами.

* **Функция расчёта WMD:** Реализована функция calculate_wmd, которая принимает два текста, преобразует их в нижний регистр, разбивает на слова и вычисляет WMD. Этот метод основан на минимальном расстоянии, необходимом для преобразования всех слов в одном документе в слова другого документа, учитывая их семантическую близость.

* **Применение WMD к данным:** Для каждой пары текстов (требования и соответствующий параграф) в DataFrame best_match_df было вычислено WMD, что позволило оценить их семантическую близость. Результаты сохранены в столбец WMD_Similarity данного DataFrame.

In [ ]:
# Загрузка предобученной модели Word2Vec
model = api.load('word2vec-google-news-300')

# Обеспечение, что модель полностью загружена как KeyedVectors для доступа к функции wmdistance
if not isinstance(model, KeyedVectors):
    model = model.wv

# Функция для вычисления WMD
def calculate_wmd(text1, text2, model):
    text1 = text1.lower().split()
    text2 = text2.lower().split()
    return model.wmdistance(text1, text2)

# Вычисление WMD для каждой пары и добавление в DataFrame
best_match_df['WMD_Similarity'] = best_match_df.apply(lambda row: calculate_wmd(row['Текст требования'], row['Текст параграфа'], model), axis=1)

# Вывод первых нескольких строк для проверки
best_match_df.head(16)


#Интерпретация значений WMD
* Большие значения WMD: Высокое значение WMD указывает на то, что тексты сильно различаются с семантической точки зрения. Слова в этих текстах находятся далеко друг от друга в векторном пространстве, что свидетельствует о большой разнице в содержании или контексте.

* Маленькие значения WMD: Низкое значение WMD означает, что тексты семантически близки. Слова из одного текста находятся близко к словам другого текста в векторном пространстве, что указывает на схожесть или релевантность тем или идей между текстами.

* Нулевое значение WMD: Если WMD равно нулю, это означает, что тексты идентичны с точки зрения используемых слов и их векторных представлений, хотя на практике такое встречается редко, если только тексты не являются полными дубликатами.